# Generación de diálogos de TV
### María Fernanda Palacio Conde

En este proyecto, usted podrá generar texto en el estilo del guión de una película usando RNNs. Se propone tomar como entrenamiento un texto como [estos](https://escribecine.com.mx/tarantino-lee-giuiones/). Puede conseguir otro de su preferencia, siendo mínimo de la misma longitud de los propuestos. La red neuronal que usted construirá, generará un guión "falso", basado en patrones que ésta reconozca en los datos de entrenamiento. 

## Obtener los datos

Guarde el texto como un archivo .txt. Abra el archivo, lea el texto.

In [ ]:
from collections import Counter

Tomaremos el guion de la película de "Toy Story 1", la cual tiene una buena longitud.

In [ ]:
"""
Cargue el archivo y escriba la ruta del mismo
"""

import helper
#TO_DO escriba la ruta de su archivo
data_dir = "/content/TOY STORY.txt"

text = helper.load_data(data_dir)

## Explorar el texto
Use `view_line_range` para observar diferentes partes de los datos. Esto le dará un sentido de la información con la que estará trabajando. Puede ver, por ejemplo que el texto está en minúscula y cada nueva línea de diálogo está separada por un caracter de nueva línea`\n`.

In [ ]:
view_line_range = (0, 10)

"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
import numpy as np

print('Estadísticas del texto')
print('Número aproximado de palabras únicas: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Número de líneas: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Promedio de palabras en cada línea: {}'.format(np.average(word_count_line)))

print('Las líneas {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Estadísticas del texto
Número aproximado de palabras únicas: 5230
Número de líneas: 6347
Promedio de palabras en cada línea: 3.3793918386639357
Las líneas 0 to 10:
                      "TOY STORY"

                   Original Story by
                     John Lasseter
                      Pete Docter
                     Andrew Stanton
                       Joe Ranft

                     Screenplay by
                      Joss Whedon


---
## Implementar funciones de preprocesamiento.
Lo primero es el preprocesamiento.  Implemente las siguientes funciones de preprocesamiento:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
Para crear un word embedding, primero se deben transformar las palabras en ids. En esta función, creamos dos diccionarios:
- Un diccionario para ir de palabras a ids, lo llamamos `vocab_to_int`
- Un diccionario para ir de ids a palabras, lo llamaremos `int_to_vocab`

Devovler estos diccionarios como una tupla **tuple** `(vocab_to_int, int_to_vocab)`

Con la ayuda de count, vamos a contar el texto que entra a la función, luego de esto lo vamos a ordenar y generar cada uno de los diccionarios.

In [ ]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Crear lookup tables para el vocabulario
    :parametro text: El texto del guión partido en palabras
    :devuelve: Una tupla de diccionarios (vocab_to_int, int_to_vocab)
    """
    # TODO: Implementar función: Utilizamos las mismas funciones que se usaron en la red LSTM
    # que completaba texto. 
    texto_contado = Counter(text)
    ordenado = sorted(texto_contado, key=texto_contado.get, reverse=True)
    int_to_vocab = {i: word for i, word in enumerate(ordenado)}
    vocab_to_int = {word: j for j, word in int_to_vocab.items()}
    print(int_to_vocab,vocab_to_int) #Imprimimos salida para saber si esta realizando bien la tarea
    # return tuple
    return (vocab_to_int, int_to_vocab)

"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
tests.test_create_lookup_tables(create_lookup_tables)

{0: 'moe_szyslak', 1: 'mike', 2: 'rotch', 3: 'you', 4: 'your', 5: 'to', 6: 'drink', 7: 'the', 8: 'yeah', 9: 'name', 10: 'on', 11: 'hey', 12: 'one', 13: "i'm", 14: 'gonna', 15: 'my', 16: 'homer', 17: 'not', 18: 'problems', 19: 'should', 20: "moe's", 21: 'tavern', 22: 'where', 23: 'elite', 24: 'meet', 25: 'bart_simpson', 26: 'eh', 27: 'hello', 28: 'is', 29: 'there', 30: 'last', 31: 'hold', 32: "i'll", 33: 'check', 34: 'has', 35: 'anybody', 36: 'seen', 37: 'lately', 38: 'listen', 39: 'little', 40: 'puke', 41: 'of', 42: 'these', 43: 'days', 44: 'catch', 45: 'and', 46: 'carve', 47: 'back', 48: 'with', 49: 'an', 50: 'ice', 51: 'pick', 52: 'whats', 53: 'matter', 54: "you're", 55: 'normal', 56: 'effervescent', 57: 'self', 58: 'homer_simpson', 59: 'i', 60: 'got', 61: 'moe', 62: 'give', 63: 'me', 64: 'another', 65: 'forget', 66: 'barney_gumble', 67: 'only', 68: 'enhance', 69: 'social', 70: 'skills'} {'moe_szyslak': 0, 'mike': 1, 'rotch': 2, 'you': 3, 'your': 4, 'to': 5, 'drink': 6, 'the': 7, 'ye

### Tokenizar puntuación
Separaremos el texto en un arreglo de palabras utilizando espacios como delimitadores.
Sine mbargo, puntuación como puntos o signos de admiración podrían crear múltiples ids para la misma palabra. Por ejemplo, "bye"  y "bye!" generarían dos ids diferentes.

Implemente la función `token_lookup` para devolver un diccionario que ser usará para tokenizar símbolos como "!", convirtiéndolo a  "||Exclamation_Mark||".  Cree un diccionario para los siguientes símbolos, endonde el símbolo es la clave (key) y el valor (value) es el token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

Este diccionario será usado para tokenizar símbolos y añadir el delimitador (espacio) a cada lado. Esto separa cada símbolo como una palabra, haciendo más fácil para la red neuronal predecir la siguiente palabra. Asegúrese de no usar un valor que pueda confundirse con una palabra: por ejemplo, en lugar de usar el valor "dash", intente algo como ||dash||.

Utilizando la referencia anterior que nos proporciona la profesora, creamos el diccionario tokenizando cada uno de los símbolos que nos podemos encontrar en nuestro guión.

In [ ]:
def token_lookup():
    """
    Generar un diccionario que convierta puntuación en tokens.
    :devuelve: Diccionario tokenizado en donde la llave (key) es el signo de puntuación y el valor es el token
    """
    # TODO: Implementar función
    token = {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||'
    }
    
    return token

"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocesar los datos y guardalos

La celda siguiente llevará a cabo el preprocesamiento de todo el texto y lo guardará en un archivo. Puede mirar el código de `preprocess_and_save_data`  en el archivo `helpers.py` para ver qué está haciendo en detalle, pero no necesita modificar dicho código.

In [ ]:
def load_preprocess():
    """
    Carga la info. de entrenamiento preprocesada y la devuelve en lotes de tamaño <batch_size> o menor
    """
    with open('preprocess.p', mode='rb') as file:
        data = pickle.load(file)
    return data

In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{0: '||return||', 1: '||period||', 2: 'the', 3: '||comma||', 4: '||exclamation_mark||', 5: 'woody', 6: 'buzz', 7: '||dash||', 8: 'to', 9: 'and', 10: 'a', 11: '||right_parentheses||', 12: '||left_parentheses||', 13: 'of', 14: '||question_mark||', 15: 'you', 16: 'his', 17: 'in', 18: 'is', 19: 'on', 20: 'up', 21: 'out', 22: 'it', 23: 'i', 24: 'toys', 25: 'sid', 26: 'head', 27: 'andy', 28: 'with', 29: 'he', 30: 'o', 31: 'back', 32: '||quotation_mark||', 33: 'from', 34: 'no', 35: 'potato', 36: 'as', 37: 'down', 38: 'for', 39: 'into', 40: 'at', 41: 'that', 42: 'are', 43: 'him', 44: 'all', 45: 'oh', 46: 'over', 47: 's', 48: 'rex', 49: 'room', 50: "andy's", 51: 'just', 52: 'mr', 53: 'what', 54: 'slinky', 55: 'int', 56: "sid's", 57: 'truck', 58: "it's", 59: 'off', 60: 'box', 61: 'get', 62: 'hey', 63: 'one', 64: 'this', 65: 'toy', 66: 'me', 67: 'car', 68: 'see', 69: 'window', 70: 'we', 71: 'door', 72: 'hannah', 73: 'can', 74: "i'm", 75: 'here', 76: 'around', 77: 'be', 78: 'now', 79: 'there', 80:

# Check Point 1
En este punto (checkpoint) usted puede descargar los archivos que se hayan generado (el texto pre-oricesado) y guardarlos localmente. Cuando regrese, cargue dichos archivos y puede comenzar aquí, de manera que no tenga que hacer el preprocesamiento del texto nuevamente. 

In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

Imprimimos los resultados del prepocesamiento para verificar que sea correcta la salida.

In [ ]:
#Observe sus diccionarios
vocab_to_int

{'||return||': 0,
 '||period||': 1,
 'the': 2,
 '||comma||': 3,
 '||exclamation_mark||': 4,
 'woody': 5,
 'buzz': 6,
 '||dash||': 7,
 'to': 8,
 'and': 9,
 'a': 10,
 '||right_parentheses||': 11,
 '||left_parentheses||': 12,
 'of': 13,
 '||question_mark||': 14,
 'you': 15,
 'his': 16,
 'in': 17,
 'is': 18,
 'on': 19,
 'up': 20,
 'out': 21,
 'it': 22,
 'i': 23,
 'toys': 24,
 'sid': 25,
 'head': 26,
 'andy': 27,
 'with': 28,
 'he': 29,
 'o': 30,
 'back': 31,
 '||quotation_mark||': 32,
 'from': 33,
 'no': 34,
 'potato': 35,
 'as': 36,
 'down': 37,
 'for': 38,
 'into': 39,
 'at': 40,
 'that': 41,
 'are': 42,
 'him': 43,
 'all': 44,
 'oh': 45,
 'over': 46,
 's': 47,
 'rex': 48,
 'room': 49,
 "andy's": 50,
 'just': 51,
 'mr': 52,
 'what': 53,
 'slinky': 54,
 'int': 55,
 "sid's": 56,
 'truck': 57,
 "it's": 58,
 'off': 59,
 'box': 60,
 'get': 61,
 'hey': 62,
 'one': 63,
 'this': 64,
 'toy': 65,
 'me': 66,
 'car': 67,
 'see': 68,
 'window': 69,
 'we': 70,
 'door': 71,
 'hannah': 72,
 'can': 73,
 

In [ ]:
int_to_vocab

{0: '||return||',
 1: '||period||',
 2: 'the',
 3: '||comma||',
 4: '||exclamation_mark||',
 5: 'woody',
 6: 'buzz',
 7: '||dash||',
 8: 'to',
 9: 'and',
 10: 'a',
 11: '||right_parentheses||',
 12: '||left_parentheses||',
 13: 'of',
 14: '||question_mark||',
 15: 'you',
 16: 'his',
 17: 'in',
 18: 'is',
 19: 'on',
 20: 'up',
 21: 'out',
 22: 'it',
 23: 'i',
 24: 'toys',
 25: 'sid',
 26: 'head',
 27: 'andy',
 28: 'with',
 29: 'he',
 30: 'o',
 31: 'back',
 32: '||quotation_mark||',
 33: 'from',
 34: 'no',
 35: 'potato',
 36: 'as',
 37: 'down',
 38: 'for',
 39: 'into',
 40: 'at',
 41: 'that',
 42: 'are',
 43: 'him',
 44: 'all',
 45: 'oh',
 46: 'over',
 47: 's',
 48: 'rex',
 49: 'room',
 50: "andy's",
 51: 'just',
 52: 'mr',
 53: 'what',
 54: 'slinky',
 55: 'int',
 56: "sid's",
 57: 'truck',
 58: "it's",
 59: 'off',
 60: 'box',
 61: 'get',
 62: 'hey',
 63: 'one',
 64: 'this',
 65: 'toy',
 66: 'me',
 67: 'car',
 68: 'see',
 69: 'window',
 70: 'we',
 71: 'door',
 72: 'hannah',
 73: 'can',
 

In [ ]:
int_text

[1565,
 1566,
 0,
 1567,
 1568,
 0,
 1049,
 1050,
 0,
 1051,
 1569,
 0,
 0,
 1570,
 113,
 0,
 1571,
 1572,
 0,
 1049,
 1050,
 0,
 1573,
 1574,
 9,
 1575,
 1576,
 0,
 0,
 32,
 65,
 1052,
 32,
 0,
 0,
 1053,
 1577,
 0,
 0,
 55,
 1,
 50,
 88,
 0,
 0,
 10,
 1578,
 13,
 124,
 237,
 1579,
 19,
 2,
 121,
 13,
 2,
 49,
 1,
 110,
 0,
 42,
 1054,
 20,
 17,
 808,
 8,
 80,
 107,
 10,
 1055,
 1056,
 809,
 1,
 0,
 2,
 88,
 18,
 1580,
 28,
 1581,
 1582,
 547,
 2,
 0,
 1583,
 13,
 417,
 1,
 0,
 0,
 63,
 13,
 2,
 237,
 140,
 10,
 1057,
 1584,
 32,
 548,
 32,
 0,
 1058,
 13,
 10,
 52,
 1,
 35,
 26,
 662,
 8,
 22,
 1,
 0,
 0,
 10,
 52,
 1,
 35,
 26,
 155,
 18,
 342,
 17,
 103,
 13,
 2,
 1058,
 1,
 2,
 0,
 108,
 46,
 13,
 27,
 3,
 10,
 1059,
 7,
 549,
 7,
 418,
 238,
 3,
 73,
 77,
 419,
 1585,
 0,
 21,
 44,
 2,
 1586,
 13,
 2,
 550,
 1,
 0,
 0,
 27,
 12,
 36,
 35,
 26,
 11,
 0,
 184,
 205,
 3,
 64,
 18,
 10,
 1060,
 7,
 0,
 20,
 4,
 89,
 1587,
 185,
 4,
 78,
 551,
 0,
 41,
 1588,
 4,
 0,
 0,
 10,
 810,
 1

## Construir la red neuronal
En esta sección usted construirá los componentes necesarios para una red neuronal recurrente, mediante la impplementación del módilo RNN y los procesos de forward y backpropagation.

### Revisar acceso a GPU

In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Entrada

Comencemos con la información preporcesada como entrada. Usaremos [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) para proveer un formato conocido a nuestro dataset, en combinación con [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), esto gestionará el batching (generación de lotes), shuffling entre otras funciones


You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implemente la función  `batch_data` para separar `words` en prociones de tamaño `batch_size` utilizando las clases `TensorDataset` y `DataLoader`.

>Usted puede usar DataLoader para generar lotes de plabras, pero dependerá de usted crear `feature_tensors` y `target_tensors` del tamaño correcto para una longitud dada de `sequence_length`.

Por ejemplo, supongamos que tenemos como entrada
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Su primer `feature_tensor` debería contener los valores:
```
[1, 2, 3, 4]
```

Y el correspondiente `target_tensor` debería símplemente ser la siguiente "palabra" (valor de la palabra tokenizada)
```
5
```
Esto debería continuar con el siguiente `feature_tensor`, `target_tensor` siendo:
```
[2, 3, 4, 5]  # features
6             # target
```

En esta sección tuve que pasar el mismo bach_size y words, debido a que mas adelante en el modelo, si habia una diferencia en los valores incurriamos en un error, por otro lado, obtuvimos el número entero de los lotes obtenibles desde el array con la ayuda de // que nos saca la parte entera.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implementar función
    #Implementamos las mismas formulas utilizadas en los talleres anteriores para
    #calcular el tamaño de batch y tambien obtener el número total de batch 
    
    # Numero total de caracteres codificados en un lote
    words = words
    total_batch_size = batch_size
    
    # Numero entero de los lotes obtenibles desde el array
    n_batches = len(words)//total_batch_size  
    
    # Cortamos los caracteres excedentes del array
    words = words[:n_batches * total_batch_size]  
    
    # Se crean las listas de feature y target
    feature, target = [], []
    target_words_length = words[:-sequence_length]
    
    # Llenar los tensores de feature y target con la lista de palabras
    for idx in range(0, len(target_words_length)):
        feature.append(words[idx: idx + sequence_length])
        target.append(words[idx + sequence_length])
        
    # Convertir las listas en tensores (feature y target)
    batch_nums = len(words) // batch_size
    feature = feature[:batch_nums * batch_size]
    target = target[:batch_nums * batch_size]

    feature_tensors = torch.from_numpy(np.asarray(feature))
    target_tensors = torch.from_numpy(np.asarray(target))
    
    # Juntar los tensores en un Dataset 
    data_set = TensorDataset(feature_tensors, target_tensors)
    
    # Se crea el dataloader y se aleatoriza la data
    data_loader = torch.utils.data.DataLoader(data_set, batch_size = batch_size, shuffle = True)
    
    # Devolver el dataloader
    return data_loader


### Testear su dataloader 

Usted tendrá que modificar este código para testear la función de batching, pero debría verse muy similar.

Abajo estamos generando algo de texto y definiento un Dataloader usando la función que usted definió rriba. Luego estamos obteniendo un lote de muestra con entradas `sample_x` y targets `sample_y` fde nuestro Dataloader.

Su código debería devolver algo como lo siguiente
```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Tamaños
Su sample_x debería ser de tamaño`(batch_size, sequence_length)` o (10, 5) en este caso, y sample_y debería sólo tener una dimensión: batch_size (10). 

### Valores
También debería usted notar que los targets, sample_y, son el *siguiente* valor en los datos ordenados test_text. entonces, para una entrada `[ 28,  29,  30,  31,  32]` que termian en el valor `32`, la salida correspondiente debería ser `33`.

In [ ]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
#sample_x, sample_y = data_iter.next()
sample_x, sample_y = next(data_iter)
print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[21, 22, 23, 24, 25],
        [14, 15, 16, 17, 18],
        [12, 13, 14, 15, 16],
        [11, 12, 13, 14, 15],
        [36, 37, 38, 39, 40],
        [ 6,  7,  8,  9, 10],
        [38, 39, 40, 41, 42],
        [32, 33, 34, 35, 36],
        [26, 27, 28, 29, 30],
        [24, 25, 26, 27, 28]])

torch.Size([10])
tensor([26, 19, 17, 16, 41, 11, 43, 37, 31, 29])


Efectivamente obtuvimos una respuesta similar al ejemplo que nos proporcina la profesora, lo cual indica que vamos por un buen camino.

---
## Construir la red neuronal
Implemente una RNN usando el módulo de PyTorch [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). Usted puede elegir una LSTM o GRU(es una variación de la LSTM). Para completar la RNN, usted deberá implementar las siguientes funciones para la clase:
 - `__init__` - Función de inicialización. 
 - `init_hidden` - Función de inicialización para el hidden state en LSTM/GRU
 - `forward` - Funcion de forward.
Recomiendo consultar el cuaderno anterior de generación de texto.

**La salida de este modelo debería ser el *último* batch de scores de palabras** luego de que una secuencia completa se ha procesado. Esto es, para una secuencia de palabras de entrada, sólo queremos como salida los scores para una única, más probable, siguiente palabra.



In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # define embedding layer        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        ## Define the LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Define the final, fully-connected output layer
        self.fc = nn.Linear(hidden_dim, output_size)

    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.fc(lstm_out)
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        # get last batch
        out = out[:, -1]

        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Definir forward y Backpropagation

Use la clase RNN que implementó para aplicar forward y backproagation. Esta función será llamada, iterativamente, en el loop de entrenamiento, como sigue:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```
Y debería devolver el error promedio sobre un lote, y el hidden state devuelto por un llamado a `RNN(inp, hidden)`. Recuerde que puede obtener este error calculándolo y llamando `loss.item()`.

**Si hay GPU disponible, debería mover los datos a esa GPU, aquí**

In [ ]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    if(train_on_gpu):
        rnn.cuda()
        
    # Creating new variables for the hidden state
    h = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()
    
    if(train_on_gpu):
        inputs, target = inp.cuda(), target.cuda()
       
    # get predicted outputs
    output, h = rnn(inputs, h)
    
    # calculate loss
    loss = criterion(output, target)
    
    #optimizer.zero_grad()
    loss.backward()
    # 'clip_grad_norm' helps prevent the exploding gradient problem in RNNs / LSTMs
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)

    optimizer.step()
    return loss.item(), h
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Entrenamiento de la red neuronal
Con la estructura de la red completa y los datos listos para proporcionárselos, es momento de entrenarla.


In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparámetros

Establezca y entrene la red neuronal con los siguientes parámetros:
- `sequence_length` el tamaño de la secuencia.
- `batch_size` cantidad de secuencias en el lote.
- `num_epochs` cantidad de épocas.
- `learning_rate` tasa de aprendizaje para un optimizador.
- `vocab_size` número de tokens únicos de nuestro vocabulario.
- `output_size` tamaño deseado de la salida.
- `embedding_dim` la dimensión de *embedding*; ha de ser menor que el vocabulario.
- `hidden_dim` la dimensión oculta de la RNN.
- `n_layers` número de capas/celdas de su RNN.
- `show_every_n_batches`  número de lotes en los que la red neuronal debe imprimir el progreso.

Si la red no está obteniendo los resultados deseados, ajuste estos parámetros y/o las capas en la clase `RNN`.


In [ ]:
# Data params
# Sequence Length
sequence_length=15
# Batch Size
batch_size =150

# data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [ ]:
# Training parameters
# Number of Epochs
num_epochs = 20
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 500
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 1 

### Entrenamiento
En la siguiente celda, entrene la red neuronal con los datos preprocesados.  Si tiene dificultades para obtener el *loss* deseado, puede cambiar los hiperparámetros. En general, se puede obtener mejores resultados con hidden_dim y n_layers más grandes, pero los modelos más grandes tardan más tiempo en entrenarse. 
> **El *loss* debe ser menor a 3.5.** 

También intente experimentar con diferentes longitudes de sucesiones, éstas determinan el tamaño de las dependencias de largo alcance que un modelo puede aprender.

In [ ]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Se han truncado las últimas 5000 líneas del flujo de salida.

Epoch:    9/20    Loss: 2.256513833999634

Epoch:    9/20    Loss: 2.208569049835205

Epoch:    9/20    Loss: 2.440105676651001

Epoch:    9/20    Loss: 2.4591639041900635

Epoch:    9/20    Loss: 2.3895421028137207

Epoch:    9/20    Loss: 2.3609063625335693

Epoch:    9/20    Loss: 2.3820035457611084

Epoch:    9/20    Loss: 2.1793763637542725

Epoch:    9/20    Loss: 2.4619088172912598

Epoch:    9/20    Loss: 2.382754325866699

Epoch:    9/20    Loss: 2.374295234680176

Epoch:    9/20    Loss: 2.370755910873413

Epoch:    9/20    Loss: 2.3239059448242188

Epoch:   10/20    Loss: 1.8749914169311523

Epoch:   10/20    Loss: 2.0247867107391357

Epoch:   10/20    Loss: 1.9657975435256958

Epoch:   10/20    Loss: 1.9239935874938965

Epoch:   10/20    Loss: 1.8967212438583374

Epoch:   10/20    Loss: 1.8339567184448242

Epoch:   10/20    Loss: 2.1450390815734863

Epoch:   10/20    Loss: 1.8668895959854126

Epoch:   10/20    Lo

### TO_DO Pregunta: ¿Cómo decidió los hiperparámetros de su modelo? 
Por ejemplo: intente con diferentes longitudes de secuencia a ver si hay un tamaño hacía que el modelo converge más rápido. ¿Qué puede decir de los parámetros hidden_dim y n_layers?

La sequence_length inicial que proporcioné es de 10 y no tuvo un buen comportamiento, por lo que aumenté a 15. Por otra parte, el batch_size lo definí en 150, lo cual es un tamaño ideal para el tamaño total del texto. num_epochs está definido como 20, pero para realizar las pruebas iniciales lo mantuve en 8. Una vez que la salida fue la esperada, aumenté a 20 para que la salida fuera mucho mejor.

Los n_layers los inicié en 2, pero al aumentar el número tuve loss más grandes, contrario a lo descrito en el enunciado. En mi caso, tuve que bajar de nuevo los n_layers. Por otra parte, hidden_dim lo fui aumentando hasta 500, ya que mejoraba el modelo.

Podemos ver la salida de cada epoch en la celda anterior, los cuales fueron muy buenos manteniendo el loss menor a 3.5.

---
# Checkpoint

Después de ejecutar la celda de entrenamiento anterior, su modelo se guardará con un nombre, `trained_rnn'. Puede descargarlo, guardarlo localmente y cargarlo cuando regrese, para no tener que entrenar cada vez.

In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

El archivo que contiene el modelo guardado está adjunto en la carpeta entregada.




## Generar el guión de TV
Use la red entrenada para generar un "falso" guión de película.

### Generar texto
Para generar el texto, la red necesita empezar con una sola palabra y repetir sus predicciones hasta que alcance una longitud determinada, para ello, utiliza la función `generate`, toma un id de palabra para empezar, `prime_id`, y genera una longitud de texto establecida, `predict_len`. 
La red también utiliza el muestreo topk para introducir algo de aleatoriedad en la elección de la siguiente palabra más probable, dado un conjunto de *scores* de palabras de salida.


In [ ]:
"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences



### Generar un Nuevo Guión
Establezca `gen_length` a la longitud del guión que desea generar y establezca `prime_word` con alguna de las palabras de su diccionario.
Es mejor empezar con un nombre para generar un guión. (También puede empezar con cualquier otro nombre que encuentre en el archivo de texto original).

Decidí iniciar el guion con Woody, un personaje principal de la película "Toy Story", lo que hace que siga el hilo de la trama. Adicionalmente, tomé como tamaño del guion 200 palabras. Con esta cantidad, ya podemos verificar si la salida tiene coherencia o, por el contrario, no tiene coherencia en la historia y en las frases.

In [ ]:
#TO_DO Establezca el tamaño del texto generado y la palabra con la que inicia el texto. 
## OJO la primera palabra debe hacer parte de su diccionario de palabras. Busque una con la cual quisiera que se empiece a generar el guión.
gen_length = 200 # modify the length to your preference
prime_word = 'woody'  # name for starting the script

"""
NO MODIFIQUE NADA A CONTINUACIÓN EN ESTA CELDA
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

woody
...

buzz and the rc car drive straight into it.

scud blindly follows straight into the closet. he clears his
wrist communicator. he crumples it up and tosses it
aside.

woody picks up woody, examines him for a beat and then smiles.

woody
(to himself)
pull my string! the mystic
portal awaits!

woody
buzz!!

buzz
i am buzz lightyear?! come in
buzz!

woody leaps off the partition and tackles buzz towards the
hallway, getting buzz's helmet.

buzz
(checking his wrist communicator)
according to place!

woody leaps off the bed and hides behind buzz.

woody
(continued)
no, no one's not a dinosaur.

sid pulls out a magnifying glass from his back pocket and
focuses the beam to enter. welcome to
attack anger. they gotta be good
to sector 12?!




Vemos que la mayor parte de la salida tiene mucho sentido con la historia de la película "Toy Story". Los diálogos se ven bien y también las expresiones de los personajes concuerdan. Adicionalmente, todos los personajes que describe el guion anterior concuerdan con los personajes de la película real.

#### Guarde su guion favorito

Cuando tenga un guion que le guste (o le parezca interesante), guárdelo en un archivo de texto.


In [25]:
# save script to a text file
f =  open("generated_script.txt","w")
f.write(generated_script)
f.close()

Por último, guardamos la salida que obtuvimos del guion, la cual me gustó mucho. La experiencia de haber generado este guion de "Toy Story" fue excelente y muy enriquecedor



# El guión NO debe ser perfecto
No pasa nada si el guión de película no tiene sentido. Simplemente debería parecer que se alternan líneas de diálogo, aquí hay un ejemplo de algunas líneas generadas.

Puede ocurrir que haya varios personajes que digan frases que casi parecen tener sentido. 
También puede tardar bastante en obtener buenos resultados y, a menudo, habrá que utilizar un vocabulario más reducido (descartando las palabras poco comunes), o conseguir más datos. 
